In [1]:
import pandas as pd
import SciServer.CasJobs as CasJobs # query with CasJobs, the primary database for the SDSS
import SciServer.SkyServer as SkyServer # show individual objects through SkyServer
import SciServer.SciDrive
import warnings
warnings.filterwarnings('ignore')

In [2]:
query = "SELECT TOP 100 \
 p.objID, p.ra, p.dec, \
 p.modelMag_r, \
 p.modelMag_u - p.modelMag_g as u_g, \
 p.modelMag_g - p.modelMag_z as g_z, \
 p.modelMag_g - p.modelMag_r as g_r, \
 p.modelMag_g - p.modelMag_i as g_i, \
 p.modelMag_r - p.modelMag_i as r_i, \
 p.modelMag_r - p.modelMag_z as r_z, \
 p.petroRad_r, p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text \
FROM\
 photoObj as p \
JOIN dbo.fGetNearbyObjEq(49.9467, 41.5131, 45) as N ON N.objID = p.objID \
WHERE\
 p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 37 \
 and p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 46 \
 and p.type = 3\
 and ((p.flags & 0x10000000) != 0)\
 and p.flags & (dbo.fPhotoFlags('MANYPETRO')\
 + dbo.fPhotoFlags('TOO_FEW_GOOD_DETECTIONS ')\
 + dbo.fPhotoFlags('BAD_MOVING_FIT_CHILD ')\
 + dbo.fPhotoFlags('PEAKS_TOO_CLOSE ')\
 + dbo.fPhotoFlags('NODEBLEND_MOVING')) = 0"
query

"SELECT TOP 100  p.objID, p.ra, p.dec,  p.modelMag_r,  p.modelMag_u - p.modelMag_g as u_g,  p.modelMag_g - p.modelMag_z as g_z,  p.modelMag_g - p.modelMag_r as g_r,  p.modelMag_g - p.modelMag_i as g_i,  p.modelMag_r - p.modelMag_i as r_i,  p.modelMag_r - p.modelMag_z as r_z,  p.petroRad_r, p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text FROM photoObj as p JOIN dbo.fGetNearbyObjEq(49.9467, 41.5131, 45) as N ON N.objID = p.objID WHERE p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 37  and p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 46  and p.type = 3 and ((p.flags & 0x10000000) != 0) and p.flags & (dbo.fPhotoFlags('MANYPETRO') + dbo.fPhotoFlags('TOO_FEW_GOOD_DETECTIONS ') + dbo.fPhotoFlags('BAD_MOVING_FIT_CHILD ') + dbo.fPhotoFlags('PEAKS_TOO_CLOSE ') + dbo.fPhotoFlags('NODEBLEND_MOVING')) = 0"

In [3]:
df0 = CasJobs.executeQuery(query, "dr16")
coords = df0[['ra', 'dec']]


In [4]:
df0

objID         ra        dec  modelMag_r       u_g       g_z  \
0   1237661055281858652  49.975071  41.210920    21.17620  0.116034  3.069038   
1   1237661055281858650  49.966287  41.204023    21.56824  0.030539  2.428406   
2   1237661055281924325  50.000579  41.146756    22.22598 -0.611626  3.441668   
3   1237661055281858616  49.947579  41.233627    22.30379 -0.125153  2.782585   
4   1237661055281858511  49.866261  41.243674    22.49465  0.549084  3.342251   
..                  ...        ...        ...         ...       ...       ...   
95  1237661083199080083  49.097482  41.313346    19.12953  2.184044  2.319084   
96  1237661083199014547  49.009436  41.364200    18.49259  2.288727  2.454182   
97  1237661083199014588  49.051821  41.356546    21.16004  1.090023  1.947895   
98  1237661083199014785  48.968554  41.474393    22.03479 -0.375448  3.478807   
99  1237661083199014840  48.990087  41.471979    22.07909  1.505632  3.074333   

         g_r       g_i       r_i       r_z  petroRad_r            flags  \
0   1.945904  2.588388  0.642485  1.123135    1.516267      68987912448   
1   1.642523  2.301208  0.658686  0.785883    1.329815   35255507616016   
2   1.860926  2.654634  0.793709  1.580742    1.623569      68987912448   
3   1.726036  2.503092  0.777056  1.056549    1.317241  105589892059480   
4   1.635235  2.619610  0.984375  1.707016    1.132399      68987912448   
..       ...       ...       ...       ...         ...              ...   
95  1.316015  1.962545  0.646530  1.003069    2.616222   35253360001296   
96  1.442841  2.004665  0.561825  1.011341    2.537312      68987912448   
97  1.009239  1.702057  0.692818  0.938656    1.461725  175990848487696   
98  1.259281  2.653374  1.394093  2.219526    1.797860  105624251793680   
99  1.654961  2.743038  1.088078  1.419373    1.223303   35253360136464   

                                            flag_text  
0                         STATIONARY BINNED1 NOPETRO   
1   DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 INT...  
2                         STATIONARY BINNED1 NOPETRO   
3   DEBLEND_NOPEAK DEBLENDED_AT_EDGE BAD_MOVING_FI...  
4                         STATIONARY BINNED1 NOPETRO   
..                                                ...  
95  DEBLENDED_AT_EDGE STATIONARY BINNED1 NOPETRO C...  
96                        STATIONARY BINNED1 NOPETRO   
97  PSF_FLUX_INTERP DEBLENDED_AT_EDGE STATIONARY B...  
98  DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY MO...  
99  DEBLENDED_AT_EDGE STATIONARY BINNED1 INTERP CO...  

[100 rows x 13 columns]